In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
import numpy as np
# Load MMLU dataset

/home/piragi/projects/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from datasets import load_dataset
tiny_data = load_dataset('tinyBenchmarks/tinyMMLU', 'all')['test']

In [3]:
tiny_data[0]

{'question': 'The number of days it takes to build a new house has a variance of 386. A sample of 40 new homes shows an average building time of 83 days. With what confidence can we assert that the average building time for a new house is between 80 and 90 days?',
 'subject': 'high_school_statistics',
 'choices': ['15.4%', '17.8%', '20.0%', '82.1%'],
 'answer': 3,
 'input_formatted': "The following are multiple choice questions (with answers) about high school statistics.\n\nWhich of the following is a correct statement about correlation?\nA. If the slope of the regression line is exactly 1, then the correlation is exactly 1.\nB. If the correlation is 0, then the slope of the regression line is undefined.\nC. Switching which variable is called x and which is called y changes the sign of the correlation.\nD. The correlation r is equal to the slope of the regression line when z-scores for the y-variable are plotted against z-scores for the x-variable.\nAnswer: D\n\nSuppose X and Y are ra

In [4]:
tiny_data['question'][0]

'The number of days it takes to build a new house has a variance of 386. A sample of 40 new homes shows an average building time of 83 days. With what confidence can we assert that the average building time for a new house is between 80 and 90 days?'

In [3]:
import gc
torch.cuda.empty_cache()
gc.collect()

0

In [5]:
first_question = tiny_data['input_formatted'][0]
# split on \n
for line in first_question.split('\n'):
    print(line)

The following are multiple choice questions (with answers) about high school statistics.

Which of the following is a correct statement about correlation?
A. If the slope of the regression line is exactly 1, then the correlation is exactly 1.
B. If the correlation is 0, then the slope of the regression line is undefined.
C. Switching which variable is called x and which is called y changes the sign of the correlation.
D. The correlation r is equal to the slope of the regression line when z-scores for the y-variable are plotted against z-scores for the x-variable.
Answer: D

Suppose X and Y are random variables with E(X) = 37, var(X) = 5, E(Y) = 62, and var(Y) = 12. What are the expected value and variance of the random variable X + Y?
A. E(X + Y) = 99, var(X + Y) = 8.5
B. E(X + Y) = 99, var(X + Y) = 13
C. E(X + Y) = 99, var(X + Y) = 17
D. There is insufficient information to answer this question.
Answer: D

After a frost warning was issued, the owner of a large orange grove asked his w

In [6]:
import torch

# Prints the number of GPUs available
print("Available GPUs:", torch.cuda.device_count())

# Lists all available GPUs
for i in range(torch.cuda.device_count()):
    print("GPU", i, ":", torch.cuda.get_device_name(i))

Available GPUs: 1
GPU 0 : NVIDIA GeForce RTX 3080


In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Load the model and tokenizer
model_name1 = "NousResearch/Hermes-2-Pro-Llama-3-8B"
model_name2 = "gradientai/Llama-3-8B-Instruct-Gradient-1048k"
tokenizer = AutoTokenizer.from_pretrained(model_name1)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
model = AutoModelForCausalLM.from_pretrained(model_name1)
model.to(device)



/home/piragi/projects/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/piragi/projects/env/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Using device: cuda


/home/piragi/projects/env/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
# Define a function to calculate the probability of each choice
def calculate_choice_probability(model, tokenizer, input_text, choice_text):
    # Concatenate the input with the choice
    full_text = input_text + " " + choice_text
    inputs = tokenizer(full_text, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits

    # Calculate the log probability of the generated tokens
    log_probs = torch.nn.functional.log_softmax(logits, dim=-1)
    # print choice
    print(choice_text)
    # Calculate the cumulative log probability of the choice tokens
    choice_tokens = tokenizer(choice_text, return_tensors="pt").input_ids.to(device)
    choice_log_prob = log_probs[0, -choice_tokens.size(1):].gather(1, choice_tokens).sum()

    return choice_log_prob.item()

# Define a function to evaluate the model on MMLU
def evaluate_model(dataset, model, tokenizer):
    model.eval()
    total = 0
    correct = 0
    
    for sample in dataset:
        input_text = sample['input_formatted']
        # choices = sample['choices']
        choices = ["A" , "B", "C", "D"]
        correct_answer = sample['answer']
        # turn correct answer into a choice 0 to A 1 to B etc
        if correct_answer == 0:
            correct_answer = "A"
        elif correct_answer == 1:
            correct_answer = "B"
        elif correct_answer == 2:
            correct_answer = "C"
        elif correct_answer == 3:
            correct_answer = "D"
            


        choice_scores = []
        for choice in choices:
            score = calculate_choice_probability(model, tokenizer, input_text, choice)
            choice_scores.append(score)
        
        # Select the choice with the highest score
        predicted_index = torch.argmax(torch.tensor(choice_scores)).item()
        predicted_answer = choices[predicted_index]
        
        # Check if the prediction is correct
        is_correct = predicted_answer == correct_answer
        if is_correct:
            correct += 1
        total += 1
        
        # Print the result
        # print("Input Text:", input_text)
        print("Predicted Answer:", predicted_answer)
        print("Correct Answer:", correct_answer)
        print("Is Correct:", is_correct)
        print()  # Print a blank line for separation
    
    accuracy = correct / total
    print(f"Accuracy: {accuracy * 100:.2f}%")

# Evaluate the model
evaluate_model(tiny_data, model, tokenizer)

A
B
C
D
Predicted Answer: D
Correct Answer: D
Is Correct: True

A
B
C
D
Predicted Answer: C
Correct Answer: C
Is Correct: True

A
B
C
D
Predicted Answer: C
Correct Answer: B
Is Correct: False

A
B
C
D
Predicted Answer: D
Correct Answer: B
Is Correct: False

A
B
C
D
Predicted Answer: C
Correct Answer: C
Is Correct: True

A
B
C
D
Predicted Answer: C
Correct Answer: C
Is Correct: True

A
B
C
D
Predicted Answer: B
Correct Answer: B
Is Correct: True

A
B
C
D
Predicted Answer: C
Correct Answer: B
Is Correct: False

A
B
C
D
Predicted Answer: C
Correct Answer: B
Is Correct: False

A
B
C
D
Predicted Answer: B
Correct Answer: D
Is Correct: False

A
B
C
D
Predicted Answer: B
Correct Answer: B
Is Correct: True

A
B
C
D
Predicted Answer: B
Correct Answer: B
Is Correct: True

A
B
C
D
Predicted Answer: D
Correct Answer: B
Is Correct: False

A
B
C
D
Predicted Answer: D
Correct Answer: D
Is Correct: True

A
B
C
D
Predicted Answer: B
Correct Answer: B
Is Correct: True

A
B
C
D
Predicted Answer: C
Correc

In [ ]:
# Define a function to evaluate the model on MMLU
def evaluate_model(dataset, model, tokenizer):
    model.eval()
    total = 0
    correct = 0
    
    for sample in dataset:
        input_text = sample['input_formatted']
        # choices = sample['choices']
        choices = ["A" , "B", "C", "D"]
        correct_answer = sample['answer']
        # turn correct answer into a choice 0 to A 1 to B etc
        if correct_answer == 0:
            correct_answer = "A"
        elif correct_answer == 1:
            correct_answer = "B"
        elif correct_answer == 2:
            correct_answer = "C"
        elif correct_answer == 3:
            correct_answer = "D"
            


        choice_scores = []
        for choice in choices:
            score = calculate_choice_probability(model, tokenizer, input_text, choice)
            choice_scores.append(score)
        
        # Select the choice with the highest score
        predicted_index = torch.argmax(torch.tensor(choice_scores)).item()
        predicted_answer = choices[predicted_index]
        
        # Check if the prediction is correct
        is_correct = predicted_answer == correct_answer
        if is_correct:
            correct += 1
        total += 1
        
        # Print the result
        # print("Input Text:", input_text)
        # print("Predicted Answer:", predicted_answer)
        # print("Correct Answer:", correct_answer)
        # print("Is Correct:", is_correct)
        # print()  # Print a blank line for separation
    
    accuracy = correct / total
    print(f"Accuracy: {accuracy * 100:.2f}%")
model_name = "NousResearch/Hermes-2-Pro-Llama-3-8B"
# model_name = "QuantFactory/Meta-Llama-3-8B-Instruct-GGUF"
# model_name = "gradientai/Llama-3-8B-Instruct-Gradient-1048k"
tokenizer = AutoTokenizer.from_pretrained(model_name)
device =  "cuda:4"
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
evaluate_model(tiny_data, model, tokenizer)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
A
B
C
D
Accuracy: 66.00%
